In [287]:
import pandas as pd
import plotly.express as px

In [288]:
path = './dados/shopping_trends.csv'
dados = pd.read_csv(path)
df = pd.DataFrame(dados)

In [289]:
df

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,Cash,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,PayPal,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Credit Card,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,PayPal,Express,No,No,24,Venmo,Weekly


# Análise de Padrões de Compra

### Categoria mais popular

In [290]:
df_produto_categoria = df.groupby('Category')[['Item Purchased']].count().sort_values('Item Purchased', ascending=False)
df_produto_categoria

,Item Purchased
Category,
Clothing,1737
Accessories,1240
Footwear,599
Outerwear,324


In [291]:
px.bar(data_frame=df_produto_categoria)

### Diferenças de comportamento entre gêneros 

In [292]:
df_categoria_genero = df[['Gender', 'Category']]

df_categoria_genero.groupby('Gender')[['Category']].value_counts()

Gender  Category   
Female  Clothing        556
        Accessories     392
        Footwear        199
        Outerwear       101
Male    Clothing       1181
        Accessories     848
        Footwear        400
        Outerwear       223
Name: count, dtype: int64

In [293]:
df_categoria_genero = df[['Gender', 'Category']]

categoria_contagem = df_categoria_genero.groupby(['Gender', 'Category']).size()
categoria_contagem = categoria_contagem.reset_index(name='Total de compras')

categoria_mais_vendida = categoria_contagem.loc[
    categoria_contagem.groupby('Gender')['Total de compras'].idxmax()
]

df_totais_genero = categoria_mais_vendida[['Gender', 'Category', 'Total de compras']]
df_totais_genero.columns = ['Gender', 'Categoria mais vendida', 'Total de compras']

total_gasto_por_genero = df.groupby('Gender')['Purchase Amount (USD)'].sum()

df_totais_genero['Total Purchase Amount (USD)'] = df_totais_genero['Gender'].map(total_gasto_por_genero)
df_totais_genero

,Gender,Categoria mais vendida,Total de compras,Total Purchase Amount (USD)
1,Female,Clothing,556,75191
5,Male,Clothing,1181,157890


### Diferenças de comportamento entre faixas etárias

In [294]:
df_categoria_idade = df[['Age', 'Category']]
df_categoria_idade.query('Age >= 18 and Age < 25')

,Age,Category
1,19,Clothing
3,21,Footwear
20,21,Clothing
24,18,Outerwear
25,18,Clothing
...,...,...
3821,20,Footwear
3830,22,Accessories
3848,22,Accessories
3868,18,Clothing


In [295]:
idades = [18, 25, 35, 50, 100] 
faixa = ['18-25', '26-35', '36-50', 'Acima de 50']

df_categoria_idade['Faixa_Etaria'] = pd.cut(df_categoria_idade['Age'], bins=idades, labels=faixa, right=False)

C:\Users\evosystem02.ti\AppData\Local\Temp\ipykernel_10452\113642023.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [296]:
categoria_por_faixa = df_categoria_idade.groupby(['Faixa_Etaria', 'Category']).size()

categoria_por_faixa = categoria_por_faixa.reset_index(name='Total de compras')

categoria_mais_comprada = categoria_por_faixa.loc[
    categoria_por_faixa.groupby('Faixa_Etaria')['Total de compras'].idxmax()
]

categoria_mais_comprada

C:\Users\evosystem02.ti\AppData\Local\Temp\ipykernel_10452\2019547693.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\evosystem02.ti\AppData\Local\Temp\ipykernel_10452\2019547693.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Faixa_Etaria,Category,Total de compras
1,18-25,Clothing,236
5,26-35,Clothing,340
9,36-50,Clothing,476
13,Acima de 50,Clothing,685


#### Exemplo de `loc`

In [297]:
df.loc[(df['Age'] > 25) & (df['Gender'] == 'Male'), ['Age', 'Gender']]

,Age,Gender
0,55,Male
2,50,Male
4,45,Male
5,46,Male
6,63,Male
...,...,...
2643,49,Male
2646,33,Male
2647,60,Male
2648,51,Male


### Quais produtos têm maior demanda em cada estação

In [298]:
df_item_season = df.groupby('Season')[['Item Purchased']].value_counts().reset_index(name='Total de Compra')

df_item_season = df_item_season.loc[
    df_item_season.groupby('Season')['Total de Compra'].idxmax()
]

df_item_season

,Season,Item Purchased,Total de Compra
0,Fall,Jacket,54
25,Spring,Sweater,52
50,Summer,Pants,50
75,Winter,Sunglasses,52


### Quantidade de clientes por Location

In [299]:
df_customer_location = df.groupby('Location')[['Customer ID']].count().sort_values('Customer ID', ascending=False)
df_customer_location = df_customer_location.rename(columns={'Customer ID': 'Customer Quantity'})
px.bar(data_frame=df_customer_location)

###  Item mais popular em cada categoria

In [300]:
df_item_categoria = (
    df.groupby('Category')['Item Purchased']
    .value_counts()
    .reset_index(name='Total de Compras')
)

df_item_categoria = df_item_categoria.sort_values(['Category', 'Total de Compras'], ascending=[True, False])

df_item_categoria

,Category,Item Purchased,Total de Compras
0,Accessories,Jewelry,171
1,Accessories,Belt,161
2,Accessories,Sunglasses,161
3,Accessories,Scarf,157
4,Accessories,Hat,154
5,Accessories,Handbag,153
6,Accessories,Backpack,143
7,Accessories,Gloves,140
8,Clothing,Blouse,171
9,Clothing,Pants,171


In [301]:
df_item_categoria_agrupado = df_item_categoria.loc[
    df_item_categoria.groupby('Category')['Total de Compras'].idxmax()
]

df_item_categoria

,Category,Item Purchased,Total de Compras
0,Accessories,Jewelry,171
1,Accessories,Belt,161
2,Accessories,Sunglasses,161
3,Accessories,Scarf,157
4,Accessories,Hat,154
5,Accessories,Handbag,153
6,Accessories,Backpack,143
7,Accessories,Gloves,140
8,Clothing,Blouse,171
9,Clothing,Pants,171


In [302]:
px.scatter(df_item_categoria, 
             x='Category', 
             y='Total de Compras', 
             color='Item Purchased', 
             title='Item Mais Vendido por Categoria',
             labels={'Total de Compras': 'Total de Compras', 'Category': 'Categoria', 'Item Purchased': 'Item'})

### Category mais comprada por Location

In [303]:
df_categoria_location = df.groupby('Location')['Category'].value_counts().reset_index(name='Total de Compras')

df_categoria_location = df_categoria_location.sort_values(['Category', 'Total de Compras'], ascending=[True, False])

df_categoria_location = df_categoria_location.loc[
    df_categoria_location.groupby('Location')['Total de Compras'].idxmax()
]

df_categoria_location = df_categoria_location.drop(columns=['Total de Compras'])

df_categoria_location

,Location,Category
0,Alabama,Clothing
4,Alaska,Clothing
8,Arizona,Clothing
12,Arkansas,Clothing
16,California,Clothing
20,Colorado,Clothing
24,Connecticut,Clothing
28,Delaware,Clothing
32,Florida,Clothing
36,Georgia,Clothing


### Purchase Amount por cada Color e Size

In [304]:
df_caracter_vendas = df[['Color','Size', 'Purchase Amount (USD)']]

df_caracter_vendas = df_caracter_vendas.groupby(['Color', 'Size'])['Purchase Amount (USD)'].sum().reset_index(name='Purchase Amount').sort_values('Purchase Amount', ascending=False)

df_caracter_vendas.head(10)

,Color,Size,Purchase Amount
89,Violet,M,5449
53,Olive,M,5121
29,Gray,M,5090
17,Charcoal,M,4661
93,White,M,4585
57,Orange,M,4513
33,Green,M,4466
77,Silver,M,4417
49,Maroon,M,4295
61,Peach,M,4249


In [305]:
df

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,Cash,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,PayPal,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Credit Card,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,PayPal,Express,No,No,24,Venmo,Weekly


### Média de avaliação por Location

In [306]:
df.groupby('Location')[['Review Rating']].mean().sort_values('Review Rating', ascending=False)

,Review Rating
Location,
Texas,3.905195
Wisconsin,3.892000
Iowa,3.847826
Maine,3.841558
California,3.834737
Michigan,3.823288
Ohio,3.814286
North Carolina,3.814103
Montana,3.808333


### Proporção de inscritos

In [307]:
df_gender_subscription = df.loc[df['Subscription Status'] == 'Yes', ['Gender', 'Purchase Amount (USD)']]

proportion_gender = df_gender_subscription['Gender'].value_counts(normalize=True)

proportion_gender.mul(100).round(2)

Gender
Male    100.0
Name: proportion, dtype: float64

In [308]:
# Somente homens são inscritos
df.loc[(df['Gender'] == 'Female') & (df['Subscription Status'] == 'Yes'), ['Gender', 'Subscription Status']]

,Gender,Subscription Status


### Proporções de Categorias

In [314]:
df_item_categoria

,Category,Item Purchased,Total de Compras
0,Accessories,Jewelry,171
1,Accessories,Belt,161
2,Accessories,Sunglasses,161
3,Accessories,Scarf,157
4,Accessories,Hat,154
5,Accessories,Handbag,153
6,Accessories,Backpack,143
7,Accessories,Gloves,140
8,Clothing,Blouse,171
9,Clothing,Pants,171


In [326]:
df_compras_categorias = df_item_categoria.groupby('Category')[['Total de Compras']].sum()
total_compras = df_compras_categorias['Total de Compras'].sum() 

df_compras_categorias['Proportion'] = (df_compras_categorias['Total de Compras'] / total_compras * 100).round(2)  
df_compras_categorias

,Total de Compras,Proportion
Category,,
Accessories,1240,31.79
Clothing,1737,44.54
Footwear,599,15.36
Outerwear,324,8.31


In [327]:
px.pie(
    df_compras_categorias, 
    values='Proportion', 
    names=df_compras_categorias.index, 
    title='Proporção de Compras por Categoria',
    color_discrete_sequence=px.colors.qualitative.Pastel 
)